In [1]:
import time
import pandas as pd
import numpy as np
import json
from pyspark.mllib.fpm import FPGrowth as FPGrowth_MLlib
from pyspark.ml.fpm import FPGrowth
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession

print('load library successfully')

load library successfully


In [2]:
conf = (SparkConf()
         .setMaster("local")
         .setAppName("My app")
         .set("spark.executor.memory", "1g"))
sc = SparkContext(conf = conf)
spark = SparkSession(sc)
# sc = SparkContext(appName="FPGrowth")
print('spark version is ', sc.version)

In [3]:
# data = sc.textFile("./data/sample_fpgrowth.txt")
# transactions = data.map(lambda line: line.strip().split(' '))
# model = FPGrowth.train(transactions, minSupport=0.2, numPartitions=10)

# data = sc.textFile("./data/samples_ttt.txt")
data = sc.textFile("./data/samples_full.txt")
transactions = data.map(lambda line: line.strip().split('###'))

print('start fp growth...')
start_t = time.time()
model = FPGrowth_MLlib.train(transactions, minSupport=0.001, numPartitions=10)
result = model.freqItemsets().collect()
for fi in result:
    print(fi)
print('cost time: ', time.time()-start_t)

start fp growth...
FreqItemset(items=['甲巯咪唑片'], freq=1340)
FreqItemset(items=['丙酸氟替卡松鼻喷雾剂'], freq=1866)
FreqItemset(items=['五子衍宗丸'], freq=3172)
FreqItemset(items=['氨甲环酸片'], freq=1699)
FreqItemset(items=['雌二醇片/雌二醇地屈孕酮片复合包装'], freq=2290)
FreqItemset(items=['沙美特罗替卡松粉吸入剂'], freq=5071)
FreqItemset(items=['磷酸奥司他韦颗粒'], freq=1065)
FreqItemset(items=['维生素D滴剂'], freq=1254)
FreqItemset(items=['足光散'], freq=992)
FreqItemset(items=['红核妇结洗液'], freq=1162)
FreqItemset(items=['丹参酮胶囊'], freq=1544)
FreqItemset(items=['维生素C加天然维生素E咀嚼片'], freq=2007)
FreqItemset(items=['人初抑菌喷雾剂'], freq=1373)
FreqItemset(items=['人初抑菌喷雾剂', '名流水晶狼牙套'], freq=1253)
FreqItemset(items=['人初抑菌喷雾剂', '名流水晶狼牙套', '天然胶乳橡胶避孕套'], freq=1252)
FreqItemset(items=['人初抑菌喷雾剂', '名流水晶狼牙套', '杜蕾斯人体润滑液 爽滑快感装'], freq=1252)
FreqItemset(items=['人初抑菌喷雾剂', '名流水晶狼牙套', '杜蕾斯人体润滑液 爽滑快感装', '天然胶乳橡胶避孕套'], freq=1252)
FreqItemset(items=['人初抑菌喷雾剂', '天然胶乳橡胶避孕套'], freq=1256)
FreqItemset(items=['人初抑菌喷雾剂', '杜蕾斯人体润滑液 爽滑快感装'], freq=1254)
FreqItemset(items=['人初抑菌喷雾剂', '杜蕾斯人体

In [4]:
def get_transactions_data():
    data_lst = []
    with open('./data/samples_full.txt', 'r', encoding='utf8') as file:
        cnt = 0
        for line in file:
            cnt += 1
            sss = line.strip().split('###')
            data_lst.append(tuple((cnt, sss)))
    return data_lst

data_lst = get_transactions_data()
for data in data_lst[:8]:
    print(data)

(1, ['清脑复神液'])
(2, ['麝香祛风湿油'])
(3, ['辅理善瞬感传感器', '健客家庭药箱'])
(4, ['益康胶囊'])
(5, ['奥氮平片'])
(6, ['氯化钾缓释片', '呋塞米片', '妇炎洁植物本草抑菌洗液', '康复新液', '螺内酯片', '花红片'])
(7, ['维生素C片', '阿胶'])
(8, ['汤臣倍健蛋白粉', '约斯夫特大片创可贴(6贴装)'])


In [5]:
df = spark.createDataFrame(data_lst, ["id", "items"])

start_t = time.time()
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.003, minConfidence=0.05)
model = fpGrowth.fit(df)
print('fp growth cost time: ', time.time()-start_t)

# Display frequent itemsets.
# model.freqItemsets.show()
model.freqItemsets.toPandas().head(50)

# Display generated association rules.
# model.associationRules.show()
model.associationRules.toPandas().head(50)

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
# model.transform(df).show()

fp growth cost time:  4.266887187957764


,antecedent,consequent,confidence,lift
0,"[漱口水, 云南白药牙膏]",[金口健牙膏益优清新],0.997736,280.488062
1,[VD钙片(中老年型)],[川贝雪梨膏],0.631319,201.939854
2,[金口健牙膏益优清新],[云南白药牙膏],0.956762,209.233885
3,[金口健牙膏益优清新],[漱口水],0.946642,252.643766
4,[咽炎片(999)],[健胃消食片],0.966049,140.955298
5,[枸橼酸西地那非片],[天然胶乳橡胶避孕套],0.078819,6.558725
6,[枸橼酸西地那非片],[进口诺丝避孕套 10只 惊喜装天然胶乳橡胶避孕套],0.275680,11.225895
7,[医用棉签],[创口贴],0.486053,65.662822
8,[医用棉签],[一次性使用口罩],0.520677,42.414132
9,"[金口健牙膏益优清新, 漱口水]",[云南白药牙膏],0.999352,218.547963


In [6]:
model.freqItemsets.toPandas().head(50)

,items,freq
0,[异维A酸软胶囊],4508
1,[五子衍宗丸],3172
2,[板蓝根颗粒],3393
3,[引阳索胶囊],3961
4,[前列安栓],2855
5,[孟鲁司特钠片],2794
6,[他克莫司软膏],3037
7,[玻璃酸钠滴眼液],3340
8,[他达拉非片],8541
9,"[他达拉非片, 进口诺丝避孕套 10只 惊喜装天然胶乳橡胶避孕套]",3116


In [8]:
start_t = time.time()
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.0005, minConfidence=0.05)
model = fpGrowth.fit(df)
print('fp growth cost time: ', time.time()-start_t)

# Display frequent itemsets.
# model.freqItemsets.show()
model.freqItemsets.toPandas().head(50)

# Display generated association rules.
# model.associationRules.show()
model.associationRules.toPandas().head(50)

print(type(model.associationRules))
model.associationRules.toPandas().to_csv('./data/associationRules_0005.csv', index=False)
model.freqItemsets.toPandas().to_csv('./data/frequentItems_0005.csv', index=False)

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
# model.transform(df).show()

fp growth cost time:  3.8965072631835938
<class 'pyspark.sql.dataframe.DataFrame'>


In [17]:
print(type(model.associationRules))
model.associationRules.toPandas().to_csv('./data/associationRules.csv', index=False)
model.freqItemsets.toPandas().to_csv('./data/frequentItems.csv', index=False)

<class 'pyspark.sql.dataframe.DataFrame'>


In [5]:
# df = sqlContext.createDataFrame(data, ["features"])
df = spark.createDataFrame([
    (0, [1, 2, 5]),
    (1, [1, 2, 3, 5]),
    (2, [1, 2])
], ["id", "items"])

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(df)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df).show()

+---------+----+
|    items|freq|
+---------+----+
|      [5]|   2|
|   [5, 2]|   2|
|[5, 2, 1]|   2|
|   [5, 1]|   2|
|      [2]|   3|
|   [2, 1]|   3|
|      [1]|   3|
+---------+----+

+----------+----------+------------------+----+
|antecedent|consequent|        confidence|lift|
+----------+----------+------------------+----+
|    [5, 2]|       [1]|               1.0| 1.0|
|    [5, 1]|       [2]|               1.0| 1.0|
|       [5]|       [2]|               1.0| 1.0|
|       [5]|       [1]|               1.0| 1.0|
|       [2]|       [5]|0.6666666666666666| 1.0|
|       [2]|       [1]|               1.0| 1.0|
|       [1]|       [5]|0.6666666666666666| 1.0|
|       [1]|       [2]|               1.0| 1.0|
|    [2, 1]|       [5]|0.6666666666666666| 1.0|
+----------+----------+------------------+----+

+---+------------+----------+
| id|       items|prediction|
+---+------------+----------+
|  0|   [1, 2, 5]|        []|
|  1|[1, 2, 3, 5]|        []|
|  2|      [1, 2]|       [5]|
+---+--

In [ ]:
def get_all_transactions():
    transactions = []
    with open('./data/orders_common_titles_info_split.json', 'r', encoding='utf8') as file:
        ddd = json.load(file)
    cnt = 0
    for code in ddd:
        # transactions.append(list(ddd[code]))
        # transactions.append([str(ele) for ele in ddd[code]])
        transactions.append(tuple([str(ele) for ele in ddd[code]]))
        cnt += 1
        if cnt > 1000:
            continue
    # print('transactions is ', transactions)
    print('len of transactions is ', len(transactions))
    return transactions

In [ ]:
transactions = get_all_transactions()